In [63]:
# !pip install kaggle

In [64]:
# from google.colab import files

# uploaded = files.upload()

# for fn in uploaded.keys():
  
#   print(
#       'User uploaded file "{name}" with length {length} bytes'.format(
#       name=fn, length=len(uploaded[fn]))
#   )
  
# # Then move kaggle.json into the folder where the API expects to find it.
# !mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

In [65]:
# !kaggle competitions download "titanic"

In [66]:
# !unzip titanic.zip

In [67]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="darkgrid")

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import StratifiedKFold

import string
import warnings
warnings.filterwarnings('ignore')

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

SEED = 42

Feature Analysis

In [68]:
print(train_df.info())
train_df.sample(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
221,222,0,2,"Bracken, Mr. James H",male,27.0,0,0,220367,13.00,NaN,S
574,575,0,3,"Rush, Mr. Alfred George John",male,16.0,0,0,A/4. 20589,8.05,NaN,S
806,807,0,1,"Andrews, Mr. Thomas Jr",male,39.0,0,0,112050,0.00,A36,S


In [69]:
print(test_df.info())
test_df.sample(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB
None


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
315,1207,3,"Hagardon, Miss. Kate",female,17.0,0,0,AQ/3. 30631,7.7333,NaN,Q
310,1202,3,"Cacic, Mr. Jego Grga",male,18.0,0,0,315091,8.6625,NaN,S
299,1191,3,"Johansson, Mr. Nils",male,29.0,0,0,347467,7.8542,NaN,S


In [70]:
# todo:
# 1.
# rewrite prepare_features for playing with features
# 2. 
# Play with features
# 3.
# Grid search on model params

# https://www.kaggle.com/code/gunesevitan/titanic-advanced-feature-engineering-tutorial
# https://machinelearningmastery.com/stacking-ensemble-machine-learning-with-python/
# https://scikit-learn.org/stable/modules/impute.html

Feature Engineering

In [71]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

# from titanic.data import SEED

target = 'Survived'


def concat_df(train_data, test_data):
    # Returns a concatenated df of training and test set
    return pd.concat([train_data, test_data], sort=True).reset_index(drop=True)


def divide_df(all_data):
    # Returns divided dfs of training and test set
    return all_data.loc[:890], all_data.loc[891:].drop(['Survived'], axis=1)


def generate_features(data_df):
    data_df['family_size'] = data_df['SibSp'] + data_df['Parch']
    data_df['name_length'] = data_df['Name'].apply(len)
    data_df['is_alone'] = 0
    data_df.loc[data_df['family_size'] == 1, 'is_alone'] = 1

    data_df['cabin'] = data_df['Cabin'].str[:1]

    data_df['title'] = 0
    data_df['title'] = data_df.Name.str.extract('([A-Za-z]+)\.')
    data_df['title'].replace(
        ['Mlle', 'Mme', 'Ms', 'Dr', 'Major', 'Lady', 'Countess', 'Jonkheer', 'Col', 'Rev', 'Capt', 'Sir', 'Don', 'Dona'],
        ['Miss', 'Miss', 'Miss', 'Mr', 'Mr', 'Mrs', 'Mrs', 'Other', 'Other', 'Other', 'Mr', 'Mr', 'Mr', 'Mrs'],
        inplace=True
    )

    data_df.loc[(data_df.Age.isnull()) & (data_df.title == 'Mr'), 'Age'] = data_df.Age[data_df.title == 'Mr'].mean()
    data_df.loc[(data_df.Age.isnull()) & (data_df.title == 'Mrs'), 'Age'] = data_df.Age[data_df.title == 'Mrs'].mean()
    data_df.loc[(data_df.Age.isnull()) & (data_df.title == 'Master'), 'Age'] = data_df.Age[
        data_df.title == 'Master'].mean()
    data_df.loc[(data_df.Age.isnull()) & (data_df.title == 'Miss'), 'Age'] = data_df.Age[data_df.title == 'Miss'].mean()
    data_df.loc[(data_df.Age.isnull()) & (data_df.title == 'Other'), 'Age'] = data_df.Age[
        data_df.title == 'Other'].mean()

    data_df.loc[data_df.Ticket.str.isdigit(), 'ticket_class'] = 1
    data_df.loc[~data_df.Ticket.str.isdigit(), 'ticket_class'] = 0
    data_df['ticket_class'] = data_df['ticket_class'].apply(int)
    
    data_df = data_df.drop(columns=['Name', 'Ticket', 'PassengerId', 'Cabin'])
    return data_df


def feature_importance(model, feature_names):
    forest_importances = pd.Series(model.feature_importances_, index=feature_names)

    fig, ax = plt.subplots()
    forest_importances.plot.bar(ax=ax)
    ax.set_title("Feature importances using MDI")
    ax.set_ylabel("Mean decrease in impurity")
    fig.tight_layout()


def generate_and_save_submission(test_df: pd.DataFrame, rez_preds, rez_file_name: str):
    submission_df = pd.DataFrame(columns=['PassengerId', 'Survived'])
    submission_df['PassengerId'] = test_df['PassengerId']
    submission_df['Survived'] = rez_preds
    submission_df.to_csv(rez_file_name, header=True, index=False)
    submission_df.head(10)


def get_feature_transformer(X: pd.DataFrame):
    X = X.copy()

    categorical_features = ['Embarked', 'Parch', 'Pclass', 'Sex', 'SibSp', 'family_size', 'is_alone', 'cabin', 'title', 'ticket_class']
    numerical_features = ['Age', 'Fare']

    categorical_pipeline = Pipeline(
        steps=[
            ("impute", SimpleImputer(strategy="most_frequent")),
            ("oh-encode", OneHotEncoder(handle_unknown="ignore", sparse=False)),
        ]
    )
    numeric_pipeline = Pipeline(
        steps=[("impute", SimpleImputer(strategy="mean")),
               ("scale", StandardScaler())]
    )
    full_transformer = ColumnTransformer(
        transformers=[
            ("numeric", numeric_pipeline, numerical_features),
            ("categorical", categorical_pipeline, categorical_features),
        ]
    )

    full_transformer.fit(X)
    return full_transformer


def prepare_features_2(X: pd.DataFrame, feature_transformer: ColumnTransformer, is_debug=True):
    if is_debug:
        print(feature_transformer.get_feature_names_out())

    X_processed = feature_transformer.transform(X)

    y = X[target]
    y_processed = SimpleImputer(strategy="most_frequent").fit_transform(
        y.values.reshape(-1, 1)
    )

    X_train, X_test, y_train, y_test = train_test_split(
        X_processed, y_processed, stratify=y_processed, random_state=SEED
    )

    return X_train, X_test, y_train, y_test


In [72]:
all_df = concat_df(train_df, test_df)
all_df = generate_features(all_df)
all_df

,Age,Embarked,Fare,Parch,Pclass,Sex,SibSp,Survived,family_size,name_length,is_alone,cabin,title,ticket_class
0,22.000000,S,7.2500,0,3,male,1,0.0,1,23,1,NaN,Mr,0
1,38.000000,C,71.2833,0,1,female,1,1.0,1,51,1,C,Mrs,0
2,26.000000,S,7.9250,0,3,female,0,1.0,0,22,0,NaN,Miss,0
3,35.000000,S,53.1000,0,1,female,1,1.0,1,44,1,C,Mrs,1
4,35.000000,S,8.0500,0,3,male,0,0.0,0,24,0,NaN,Mr,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,32.545531,S,8.0500,0,3,male,0,NaN,0,18,0,NaN,Mr,0
1305,39.000000,C,108.9000,0,1,female,0,NaN,0,28,0,C,Mrs,0
1306,38.500000,S,7.2500,0,3,male,0,NaN,0,28,0,NaN,Mr,0
1307,32.545531,S,8.0500,0,3,male,0,NaN,0,19,0,NaN,Mr,1


In [73]:
all_df.groupby('Survived')['Survived'].count()

Survived
0.0    549
1.0    342
Name: Survived, dtype: int64

Model Engineering

In [74]:
df_train, df_test = divide_df(all_df)

In [78]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

feature_transformer = get_feature_transformer(df_train)
print(feature_transformer.get_feature_names_out())
X_train = feature_transformer.transform(df_train)
y_train = df_train['Survived'].values
X_test = feature_transformer.transform(df_test)

model = RandomForestClassifier(random_state=SEED)
model.fit(X_train, y_train)
print(model)

preds = model.predict(X_train)
acc = accuracy_score(y_train, preds)
print('accuracy_score: ', acc)

rez_preds = model.predict(X_test)
y_pred = rez_preds.astype(int)
submission_df = pd.DataFrame(columns=['PassengerId', 'Survived'])
submission_df['PassengerId'] = test_df['PassengerId']
submission_df['Survived'] = y_pred
submission_df.to_csv('submissions_11.csv', header=True, index=False)
submission_df.head(10)

['numeric__Age' 'numeric__Fare' 'categorical__Embarked_C'
 'categorical__Embarked_Q' 'categorical__Embarked_S'
 'categorical__Parch_0' 'categorical__Parch_1' 'categorical__Parch_2'
 'categorical__Parch_3' 'categorical__Parch_4' 'categorical__Parch_5'
 'categorical__Parch_6' 'categorical__Pclass_1' 'categorical__Pclass_2'
 'categorical__Pclass_3' 'categorical__Sex_female' 'categorical__Sex_male'
 'categorical__SibSp_0' 'categorical__SibSp_1' 'categorical__SibSp_2'
 'categorical__SibSp_3' 'categorical__SibSp_4' 'categorical__SibSp_5'
 'categorical__SibSp_8' 'categorical__family_size_0'
 'categorical__family_size_1' 'categorical__family_size_2'
 'categorical__family_size_3' 'categorical__family_size_4'
 'categorical__family_size_5' 'categorical__family_size_6'
 'categorical__family_size_7' 'categorical__family_size_10'
 'categorical__is_alone_0' 'categorical__is_alone_1'
 'categorical__cabin_A' 'categorical__cabin_B' 'categorical__cabin_C'
 'categorical__cabin_D' 'categorical__cabin_E' 'c

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,1
4,896,0
5,897,0
6,898,0
7,899,0
8,900,1
9,901,0


Predictions